# Number partitioning

In this notebook we study the general problem of number partitioning. The goal of this discrete optimization problem is to partition a set $S$ of $N$ positive numbers, $S = \{s_1,\dots,s_N\}$ into two disjoint subsets $R$ and $S-R$ whose elements add to the same value,

\begin{equation}
\sum_{s\in R} s = \sum_{s\in S-R} s. 
\end{equation}

Its decision form, i.e., "is there a partition of $S$ such that the elements R and S-R have the same sum?" is NP-complete. 

## Ising formulation
Consider the following Ising Hamiltonian. 

\begin{equation}
H = A\bigg(\sum_{i=1}^N n_i s_i\bigg)^2, \qquad s_i = \pm 1.
\end{equation}

If there is a solution to this partitioning problem, then the ground state energy of the Hamiltonian above is zero and, conversely, if the ground energy of the Hamiltonian above is zero, the sets $R = \{n_i\in S |s_i=1\}$ and $S-R = \{n_i\in S |s_i=-1\}$  is a solution to this partitioning problem. Thus this problem can very easily be framed in terms of an Ising model. Expanding the above equation, and noting that $s_i^2 = 1$, we get

\begin{equation}
H = A\bigg(\sum_{j>i}^N 2n_in_j s_is_j\bigg)+ AB
\end{equation}

with 

\begin{equation}
B = \sum_{i=1}^N n_i^2
\end{equation}



We refer the reader to [Lucas](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full) for a discussion of the Ising implementation of this problem. Below We code a function that returns the symmetric, quadratic form $n_in_j$ as a dictionary.

In [38]:
def get_quadratic(int_array,scale=1):
    int_array = np.sort(int_array)
    quadratic = {(i,j): scale*2*n_i*n_j for j, n_j in enumerate(int_array) for i, n_i in enumerate(int_array) if j>i}
    offset = scale*np.sum(int_array**2)
    return offset, quadratic 

## Classical implementation

We start with the classical implementation using the [dimod](https://docs.ocean.dwavesys.com/projects/dimod/en/master/reference/index.html) module. Let us consider the simple case where $S = \{0,1,2,3,4,5,6,7,8,9,10,11\}$. It is clear that a solution exists, for example $R = \{0,2,4,7,9,11\}$ and $S-R = \{1,3,5,6,8,10\}$.

In [39]:
#linear = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11}
linear = {}
S = [0,1,2,3,4,5,6,7,8,9,10,11]
offset,quadratic = get_quadratic(S)
vartype = dimod.Vartype.SPIN
ham = dimod.BinaryQuadraticModel(linear,quadratic,offset,dimod.Vartype.SPIN)



#### Solving with exact sampler
The dimod module uses [samplers](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/intro/intro_samplers.html#) to find the optimal solution of a binary optimization problem. As an example, let us find the optimal partition of $S = \{0,1,2,3,4,5,6,7,8,9,10,11\}$ using the `ExactSolver` sampler.

In [41]:
def sample(ham,sampler=None, ascending=True,sample_kw=None):
    if sampler is None:
        sampler = dimod.ExactSolver()
    df = sampler.sample(ham).to_pandas_dataframe() 
    df.sort_values(by=['energy'], axis=0,ascending=ascending,inplace=True)
    return df
    
    
    

#### sampling

In [45]:
%%time
samples = sample(ham)
samples

CPU times: user 10 ms, sys: 951 µs, total: 11 ms
Wall time: 6.37 ms


,0,1,2,3,4,5,6,7,8,9,10,11,energy,num_occurrences
1864,-1,-1,1,1,-1,1,1,1,-1,-1,1,-1,0.0,1
1263,-1,-1,-1,1,1,-1,-1,1,-1,1,1,-1,0.0,1
3317,1,1,1,1,-1,-1,-1,1,-1,1,-1,1,0.0,1
3316,-1,1,1,1,-1,-1,-1,1,-1,1,-1,1,0.0,1
2114,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4096.0,1
2731,-1,1,1,1,1,1,1,1,1,1,1,1,4356.0,1
2730,1,1,1,1,1,1,1,1,1,1,1,1,4356.0,1
1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4356.0,1


In [47]:
offset

506

**Observations:** The `ExactSolver` exhausts the space of solutions, as is visible from from the 4096($=2^{12}$) solutions found. Evidently, the full set of solutions for a list of $N$ numbers contains $2^N$ solutions, so this exhaustive approach quickly becomes impracticable.

## Quantum implementation

To run this code in aws, we need to [sign in](https://197581744476.signin.aws.amazon.com/console) with our user credentials.

In [46]:
ae
exact_sampler = dimod.ExactSolver()
all_samples = exact_sampler.sample(ham)
len(all_samples)

NameError: name 'ae' is not defined

In [25]:
all_samples

SampleSet(rec.array([([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 2431., 1),
           ([ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 2431., 1),
           ([ 1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 2301., 1),
           ...,
           ([ 1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1], 1135., 1),
           ([ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1], 1221., 1),
           ([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1], 1221., 1)],
          dtype=[('sample', 'i1', (12,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], {}, 'SPIN')

In [12]:
dir(part)

['__abstractmethods__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_adj',
 '_asdict',
 '_init_bqm',
 '_init_components',
 '_init_number',
 '_offset',
 '_vartype',
 'add_interaction',
 'add_interactions_from',
 'add_offset',
 'add_variable',
 'add_variables_from',
 'adj',
 'base',
 'binary',
 'change_vartype',
 'contract_variables',
 'copy',
 'degree',
 'degrees',
 'dtype',
 'empty',
 'energies',
 'energy',
 'fix_variable',
 'fix_variables',
 'flip_variable',
 'from_coo',
 'from_ising',
 'from_networkx_graph',
 'from_numpy_matrix',
 'from_numpy_vectors',
 'from_qubo',
 'from_seriali

In [13]:
part.energy()

TypeError: energy() missing 1 required positional argument: 'sample'

In [1]:
conda install networkx

Solving environment: done

## Package Plan ##

  environment location: /home/rio/anaconda3/envs/dwave

  added / updated specs:
    - networkx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    networkx-2.5               |             py_0         1.1 MB
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following NEW packages will be INSTALLED:

  networkx           pkgs/main/noarch::networkx-2.5-py_0



networkx-2.5         | 1.1 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [9]:
%%time
import numpy as np
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
#%matplotlib inline
import networkx as nx
from collections import defaultdict
from itertools import combinations
import time
import dwave_networkx as dnx
from dimod.binary_quadratic_model import BinaryQuadraticModel
#import dimod

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 17.4 µs


In [7]:
dir(dimod.binary_quadratic_model)

['AdjDictBQM',
 'BQM',
 'BinaryQuadraticModel',
 'Container',
 'Iterable',
 'Sized',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'deserialize_ndarrays',
 'inspect',
 'iter_serialize_variables',
 'np',
 'serialize_ndarrays',
 'warnings']

In [3]:
!which pip

/home/rio/anaconda3/envs/dwave/bin/pip


In [4]:
conda install dimod

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - dimod

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [5]:
! pip install dimod